## Trabajo práctico 3: Python
Ejecuto el código y escribo tantas palabras como se desee, cuando se quiera finalizar la ejecución, escribo la palabra: close

Nota: Si se envía la palabra “close” en medio de una oración, se procesarán  todas las palabras que se encuentran antes que “close”, una vez que se procese esta palabra, se finaliza la ejecución


In [1]:
import multiprocessing

def father():
    send, recive = multiprocessing.Pipe()

    process = multiprocessing.Process(target=child, args=(send, recive))
    process.start()

    end_process = False
    while end_process == False:
        text = input("Ingrese el texto a procesar: ").split()
        for word in text:
            if word != "close":
                send.send(word)
            else:
                end_process = True;
                break;

    send.send("close")
    process.join()
    if recive.poll():
        stats = recive.recv()
        print("Cantidad de caracteres totales: ", stats["total_length"])
        print("Cantidad de letras: ", stats["total_letters"])
        print("Cantidad de dígitos: ", stats["total_digits"])
        print("La palabra de mayor longitud: ", stats["longest_word"])
        print("La palabra de menor longitud: ", stats["shortest_word"])

    send.close()
    recive.close()

def child(send_pipe, recieve_pipe):
    text = recieve_pipe.recv()

    stats = {
        "total_letters": 0,
        "total_digits": 0,
        "total_length": 0,
        "longest_word":  text if text != "close" else "",
        "shortest_word": text if text != "close" else ""
    }

    while text != "close":
        word_length = len(text)

        if word_length > len(stats["longest_word"]):
            stats["longest_word"] = text
        elif word_length < len(stats["shortest_word"]):
            stats["shortest_word"] = text

        for char in text:
            if char.isalpha():
                stats["total_letters"] += 1
            elif char.isdigit():
                stats["total_digits"] += 1

        stats["total_length"] += word_length
        text = recieve_pipe.recv()

    send_pipe.send(stats)

if __name__ == '__main__':
    father()


Ingrese el texto a procesar: hola como estas este es un texto de prueba para ver la asignacion de palabras en un texto si escribo que tengo 25 años deberia calcular
Ingrese el texto a procesar: que naci en 1998
Ingrese el texto a procesar: close
Cantidad de caracteres totales:  123
Cantidad de letras:  117
Cantidad de dígitos:  6
La palabra de mayor longitud:  asignacion
La palabra de menor longitud:  es


Conclusión: Para la utilización del pipe no hubo ningún inconveniente, lo que mas llevo tiempo fue determinar que enviar por el pipe, en este caso fue un diccionario el cual actualizo en el proceso hijo y lo envío al proceso padre una vez que se quiera finalizar la ejecución, para que este pueda mostrarlo por pantalla.